In [ ]:
import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf

import pecanpy

In [ ]:
%matplotlib inline
plt.style.use("ggplot")

# Motivation

Want to train a model to forecast individual household electricity usage at forecast horizons as short as 5 minutes ahead.  This forecasting model will be used to generate bids (and possibly offers!) for submission to an electricity auction.

In [ ]:
# define user credentials
USER_NAME = ???
PASSWORD = ???

# define db server params
SCHEMA = ???
HOST = ???
PORT = ???
DB = ???

# create the engine that connects to the database...
engine = pecanpy.create_engine(USER_NAME, PASSWORD, HOST, PORT, DB)

In [ ]:
with engine.connect() as con:
    metadata_df = pecanpy.read_metadata_table(con, schema=SCHEMA)

In [ ]:
metadata_df.head()

Will demonstrate how to build various RNN architectures to forecast indidividual demand for electricity. 

* Pure consumer
* Prosumer

Want to use households for which substantial data exists...

In [ ]:
at_least_fours_years_egauge_data = (metadata_df.egauge_min_time <= "2014-01-01") & (metadata_df.egauge_max_time >= "2018-01-01")
is_prosumer = metadata_df.pv
consumers = metadata_df[["egauge_min_time", "egauge_max_time"]][at_least_fours_years_egauge_data & (~is_prosumer)]
prosumers = metadata_df[["egauge_min_time", "egauge_max_time"]][at_least_fours_years_egauge_data & is_prosumer]

In [ ]:
consumers

In [ ]:
prosumers

## Read data on a random prosumer from the database

In [ ]:
# randomly select a prosumer
time = datetime.datetime.now()
seed = time.hour * 10000 + time.minute * 100 + time.second
prng = np.random.RandomState(seed)
prosumer = prosumers.sample(n=1, random_state=prng).iloc[0, :]

# alternatively, can use prosumer with dataid=8317 (very few missing obs!)
prosumer = prosumers.loc[8317, :]

In [ ]:
with engine.connect() as con:

    # download several years worth of data
    start_time = pd.Timestamp("2014-01-01", tz="US/Central", freq='T')
    end_time = pd.Timestamp("2018-01-01", tz="US/Central", freq='T')
    dataid = prosumer.name
    prosumer_egauge_df = pecanpy.read_electricity_egauge_query(con, SCHEMA, dataid, start_time, end_time, "all", 'T')


In [ ]:
# pickle the data to disk for safe keeping!
prosumer_egauge_df.to_pickle("./prosumer_egauge_data_{}.pkl".format(prosumer.name))

In [ ]:
prosumer_egauge_df.shape

## Read data on a random consumer from the database

In [ ]:
time = datetime.datetime.now()
seed = time.hour * 10000 + time.minute * 100 + time.second
prng = np.random.RandomState(seed)
consumer = consumers.sample(n=1, random_state=prng).iloc[0, :]

# alternative use consumer with dataif=3392
consumer = consumers.loc[3392, :]

In [ ]:
with engine.connect() as con:

    # download several years worth of data
    start_time = pd.Timestamp("2014-01-01", tz="US/Central", freq='T')
    end_time = pd.Timestamp("2018-01-01", tz="US/Central", freq='T')
    dataid = consumer.name
    consumer_egauge_df = pecanpy.read_electricity_egauge_query(con, SCHEMA, dataid, start_time, end_time, "all", 'T')


In [ ]:
# pickle the data to disk for safe keeping!
consumer_egauge_df.to_pickle("./consumer_egauge_data_{}.pkl".format(consumer.name))

In [ ]:
consumer_egauge_df.shape

## Data preprocessing

Need to do a bit of preprocessing of the data before we are ready to train our RNN models.

### Re-indexing the DataFrames

In [ ]:
new_index = pd.date_range(start=start_time,
                          end=end_time,
                          freq='T',
                          tz="US/Central",
                          closed="left")
reindexed_prosumer_egauge_df = prosumer_egauge_df.reindex(index=new_index, method="ffill")
reindexed_consumer_egauge_df = consumer_egauge_df.reindex(index=new_index, method="ffill")

In [ ]:
reindexed_consumer_egauge_df.use.plot()

In [ ]:
reindexed_prosumer_egauge_df.grid.plot()

In [ ]:
# should we consider standardizing variables?
reindexed_consumer_egauge_df.describe()

In [ ]:
reindexed_prosumer_egauge_df.describe()

### Create the target variables

In [ ]:
# prediction target is net energy demand from the grid at some point in the future
forecast_horizon = 5
consumer_target = reindexed_consumer_egauge_df.grid.shift(periods=-forecast_horizon)
prosumer_target = reindexed_prosumer_egauge_df.grid.shift(periods=-forecast_horizon)

In [ ]:
# now shift all of the features forward to align the timestamp of the features with that of the target
reindexed_consumer_egauge_df["target"] = consumer_target
reindexed_prosumer_egauge_df["target"] = prosumer_target

### Drop or fill all remaining NaNs

In [ ]:
processed_consumer_egauge_df = (reindexed_consumer_egauge_df.fillna(axis=0, method="ffill")
                                                            .dropna(axis=1, how="any")
                                                            .drop(axis=1, labels=["dataid"]))

processed_prosumer_egauge_df = (reindexed_prosumer_egauge_df.fillna(axis=0, method="ffill")
                                                            .dropna(axis=1, how="any")
                                                            .drop(axis=1, labels=["dataid"]))

In [ ]:
processed_consumer_egauge_df.head()

In [ ]:
processed_prosumer_egauge_df.head()

In [ ]:
# shape is (n_obs, n_inputs + 1)
processed_consumer_egauge_df.shape

In [ ]:
processed_prosumer_egauge_df.shape

## Train, validation, and testing split

In [ ]:
def _train_test_split(df, timestamp):
    train_idxs = df.index < timestamp
    test_idxs = df.index >= timestamp
    
    # split the input dataframe into training and testing sets
    training_df = df.loc[train_idxs]
    testing_df = df.loc[test_idxs]
    
    return training_df, testing_df


def _train_validation_test_split(df, timestamp1, timestamp2):
    assert timestamp1 < timestamp2
    train_idxs = df.index < timestamp1
    validation_idxs = (df.index >= timestamp1) & (df.index < timestamp2)
    test_idxs = df.index >= timestamp2
    
    # split the input dataframe into training, validation, and testing sets
    training_df = df.loc[train_idxs]
    validation_df = df.loc[validation_idxs]
    testing_df = df.loc[test_idxs]
    
    return training_df, validation_df, testing_df


In [ ]:
# train the model on first 3 year of data and use the final year as the testing data
n_time_steps = 24 * 60
training_end_time = start_time + (365 + 365 + 366) * n_time_steps
consumer_training_df, consumer_testing_df = _train_test_split(processed_consumer_egauge_df, training_end_time)
prosumer_training_df, prosumer_testing_df = _train_test_split(processed_prosumer_egauge_df, training_end_time)

In [ ]:
# len of three splits must be multiples of n_timesteps!
assert len(consumer_training_df) % n_time_steps == 0
assert len(consumer_testing_df) % n_time_steps == 0

assert len(prosumer_training_df) % n_time_steps == 0
assert len(prosumer_testing_df) % n_time_steps == 0

In [ ]:
prosumer_training_df.head()

In [ ]:
consumer_training_df.tail()

In [ ]:
consumer_training_features_df = consumer_training_df.drop(axis=1, labels="target", inplace=False)                                
consumer_training_target = consumer_training_df.target

consumer_testing_features_df = consumer_testing_df.drop(axis=1, labels="target", inplace=False)
consumer_testing_target = consumer_testing_df.target

prosumer_training_features_df = prosumer_training_df.drop(axis=1, labels="target", inplace=False)                                
prosumer_training_target = prosumer_training_df.target

prosumer_testing_features_df = prosumer_testing_df.drop(axis=1, labels="target", inplace=False)
prosumer_testing_target = prosumer_testing_df.target

In [ ]:
def _fetch_next_training_batch(training_features_df, training_target, batch_idx, batch_size, n_steps, n_inputs, n_outputs):
    start = batch_idx * batch_size * n_steps
    stop = start + batch_size * n_steps
    X = training_features_df.values[start:stop]
    y = training_target.values[start:stop]
    return X.reshape(-1, n_steps, n_inputs), y.reshape(-1, n_steps, n_outputs)


## Basic RNN approach

In [ ]:
tf.placeholder?

In [ ]:
def basic_rnn(n_inputs, n_outputs=1):
    graph = tf.Graph()
    X = tf.placeholder(tf.float64, [None, n_time_steps, n_inputs])
    y = tf.placeholder(tf.float64, [None, n_time_steps, n_outputs])

    # multiple RNN layers
    n_layers = 2
    n_neurons = 20
    rnn_layers = [tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.tanh) for layer in range(n_layers)]
    multi_layer_cell = tf.contrib.rnn.MultiRNNCell(rnn_layers) 
    rnn_outputs, rnn_states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype=tf.float64)

    # use a single dense layer to reduce the dimensionality
    stacked_rnn_outputs = tf.reshape(rnn_outputs, [-1, n_neurons])
    stacked_outputs = tf.layers.dense(stacked_rnn_outputs, n_outputs)
    y_hat = tf.reshape(stacked_outputs, [-1, n_time_steps, n_outputs])

    # define the loss function and an optimizer
    error = y_hat - y
    mean_square_error = tf.reduce_mean(tf.square(error))
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(mean_square_error, name="training_op")

    # create a saver to store model output
    saver = tf.train.Saver()

    init = tf.global_variables_initializer()

In [ ]:
basic_rnn_graph.

In [ ]:
n_training_epochs = 150

batch_size = 100
n_training_obs, _ = consumer_training_df.shape 
n_training_batches = ((n_training_obs // n_time_steps) // batch_size) + 1

basic_rnn_graph = basic_rnn(consumer_training_features_df.shape[1])

with tf.Session(basic_rnn_graph) as sess:
    init.run()
    for i in range(n_training_epochs):
        for j in range(n_training_batches):
            X_batch, y_batch = _fetch_next_training_batch(consumer_training_features_df, consumer_training_target, j, batch_size, n_time_steps, n_inputs, n_outputs)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            loss = mean_square_error.eval(feed_dict={X: X_batch, y: y_batch})
        if i % 10 == 0:
            print("After {} training epochs, the MSE on most recent batch is {}.".format(i, loss))

    saver.save(sess, "./trained-models/basic_rnn_load_forecasting_model_for_{}".format(consumer.name))

In [ ]:
tf.reset_default_graph()
tf.set_random_seed(42)

# placeholders to fill with data eventually
_, n_cols = consumer_training_df.shape
n_outputs = 1
n_inputs = n_cols - n_outputs

X = tf.placeholder(tf.float64, [None, n_time_steps, n_inputs])
y = tf.placeholder(tf.float64, [None, n_time_steps, n_outputs])

# multiple RNN layers
n_layers = 2
n_neurons = 20
rnn_layers = [tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.tanh) for layer in range(n_layers)]
multi_layer_cell = tf.contrib.rnn.MultiRNNCell(rnn_layers) 
rnn_outputs, rnn_states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype=tf.float64)

# use a single dense layer to reduce the dimensionality
stacked_rnn_outputs = tf.reshape(rnn_outputs, [-1, n_neurons])
stacked_outputs = tf.layers.dense(stacked_rnn_outputs, n_outputs)
y_hat = tf.reshape(stacked_outputs, [-1, n_time_steps, n_outputs])

# define the loss function and an optimizer
error = y_hat - y
mean_square_error = tf.reduce_mean(tf.square(error))
optimizer = tf.train.AdamOptimizer()
training_op = optimizer.minimize(mean_square_error, name="training_op")

# create a saver to store model output
saver = tf.train.Saver()

init = tf.global_variables_initializer()

In [ ]:
n_training_epochs = 150

batch_size = 100
n_training_obs, _ = consumer_training_df.shape 
n_training_batches = ((n_training_obs // n_time_steps) // batch_size) + 1

with tf.Session() as sess:
    init.run()
    for i in range(n_training_epochs):
        for j in range(n_training_batches):
            X_batch, y_batch = _fetch_next_training_batch(consumer_training_features_df, consumer_training_target, j, batch_size, n_time_steps, n_inputs, n_outputs)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            loss = mean_square_error.eval(feed_dict={X: X_batch, y: y_batch})
        if i % 10 == 0:
            print("After {} training epochs, the MSE on most recent batch is {}.".format(i, loss))

    saver.save(sess, "./trained-models/basic_rnn_load_forecasting_model_for_{}".format(consumer.name))

In [ ]:
with tf.Session() as sess:                          
    saver.restore(sess, "./trained-models/basic_rnn_load_forecasting_model_for_{}".format(consumer.name))
    X_test = consumer_testing_features_df.values
    y_test = consumer_testing_target.values
    rnn_predictions, rnn_mse = sess.run([y_hat, mean_square_error], feed_dict={X: X_test.reshape(-1, n_time_steps, n_inputs), y: y_test.reshape(-1, n_time_steps, n_outputs)})

In [ ]:
rnn_mse

In [ ]:
testing_predictions = pd.Series(rnn_predictions.ravel(),
                                index=consumer_testing_target.index,
                                name="predictions")
rnn_results_df = pd.concat([consumer_testing_target, testing_predictions], join="inner", axis=1)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10,8))
rnn_results_df.head(60 * 24).plot(ax=ax);

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.ma.masked_invalid(np.abs((y_true - y_pred) / y_true)).mean() * 100

In [ ]:
mean_absolute_percentage_error(rnn_results_df.target, rnn_results_df.predictions)

# Long Short-Term Memory (LSTM) Units

![A Long Short-Term Memory (LSTM) Unit](./assets/greff_lstm_diagram.png)

Image by Klaus Greff and colleagues as published in [*LSTM: A Search Space Odyssey*](https://arxiv.org/abs/1503.04069).

In [ ]:
tf.reset_default_graph()
tf.set_random_seed(42)

# placeholders to fill with data eventually
X = tf.placeholder(tf.float64, [None, n_time_steps, n_inputs])
y = tf.placeholder(tf.float64, [None, n_time_steps, n_outputs])

# multiple LSTM layers with peep-hole connections
lstm_layers = [tf.contrib.rnn.LSTMCell(num_units=n_neurons, use_peepholes=True) for layer in range(n_layers)]
multi_layer_cell = tf.contrib.rnn.MultiRNNCell(lstm_layers) 
lstm_outputs, lstm_states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype=tf.float64)

# use a single dense layer to reduce the dimensionality
stacked_lstm_outputs = tf.reshape(lstm_outputs, [-1, n_neurons])
stacked_outputs = tf.layers.dense(stacked_lstm_outputs, n_outputs)
y_hat = tf.reshape(stacked_outputs, [-1, n_time_steps, n_outputs])

# define the loss function and an optimizer
error = y_hat - y
mean_square_error = tf.reduce_mean(tf.square(error))
optimizer = tf.train.AdamOptimizer()
training_op = optimizer.minimize(mean_square_error)

# create a saver to store model output
saver = tf.train.Saver()

init = tf.global_variables_initializer()

In [ ]:
with tf.Session() as sess:
    init.run()
    for i in range(n_training_epochs):
        for j in range(n_training_batches):
            X_batch, y_batch = _fetch_next_training_batch(j, batch_size, n_time_steps, n_inputs, n_outputs)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            loss = mean_square_error.eval(feed_dict={X: X_batch, y: y_batch})
        if i % 10 == 0:
            print("After {} training epochs, the MSE on the most recent batch is {}.".format(i, loss))

    saver.save(sess, "./lstm_rnn_load_forecasting_model_for_{}".format(dataid))

In [ ]:
with tf.Session() as sess:                          
    saver.restore(sess, "./lstm_rnn_load_forecasting_model_for_{}".format(dataid))
    X_test = testing_features_df.values
    y_test = testing_target.values
    lstm_predictions, lstm_mse = sess.run([y_hat, mean_square_error], 
                                          feed_dict={X: X_test.reshape(-1, n_time_steps, n_inputs), y: y_test.reshape(-1, n_time_steps, n_outputs)})

In [ ]:
lstm_mse

In [ ]:
testing_predictions = pd.Series(lstm_predictions.ravel(),
                                index=testing_target.index,
                                name="predictions")
lstm_results_df = pd.concat([testing_target, testing_predictions], join="inner", axis=1)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10,8))
lstm_results_df.head(60 * 24).plot(ax=ax);

In [ ]:
mean_absolute_percentage_error(lstm_results_df.target, lstm_results_df.predictions)

# Gated Recurrent Units (GRU)

![A Long Short-Term Memory (LSTM) Unit](./assets/lstm_gru.png)

Image by Klaus Greff and colleagues as published in [*Empirical Evaluation of Gated Recurrent Neural Networks on Sequence Modeling*](https://arxiv.org/abs/1412.3555). 

In [ ]:
tf.reset_default_graph()
tf.set_random_seed(42)

# placeholders to fill with data eventually
X = tf.placeholder(tf.float64, [None, n_time_steps, n_inputs])
y = tf.placeholder(tf.float64, [None, n_time_steps, n_outputs])

# multiple GRU layers
gru_layers = [tf.contrib.rnn.GRUCell(num_units=n_neurons) for layer in range(n_layers)]
multi_layer_cell = tf.contrib.rnn.MultiRNNCell(gru_layers) 
gru_outputs, gru_states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype=tf.float64)

# use a single dense layer to reduce the dimensionality
stacked_gru_outputs = tf.reshape(gru_outputs, [-1, n_neurons])
stacked_outputs = tf.layers.dense(stacked_gru_outputs, n_outputs)
y_hat = tf.reshape(stacked_outputs, [-1, n_time_steps, n_outputs])

# define the loss function and an optimizer
error = y_hat - y
mean_square_error = tf.reduce_mean(tf.square(error))
optimizer = tf.train.AdamOptimizer()
training_op = optimizer.minimize(mean_square_error)

# create a saver to store model output
saver = tf.train.Saver()

init = tf.global_variables_initializer()

In [ ]:
with tf.Session() as sess:
    init.run()
    for i in range(n_training_epochs):
        for j in range(n_training_batches):
            X_batch, y_batch = _fetch_next_training_batch(j, batch_size, n_time_steps, n_inputs, n_outputs)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            loss = mean_square_error.eval(feed_dict={X: X_batch, y: y_batch})
        if i % 10 == 0:
            print("After {} training epochs, the MSE on the most recent batch is {}.".format(i, loss))

    saver.save(sess, "./gru_rnn_load_forecasting_model_for_{}".format(dataid))

In [ ]:
with tf.Session() as sess:                          
    saver.restore(sess, "./gru_rnn_load_forecasting_model_for_{}".format(dataid))
    X_test = testing_features_df.values
    y_test = testing_target.values
    gru_predictions, gru_mse = sess.run([y_hat, mean_square_error],
                                        feed_dict={X: X_test.reshape(-1, n_time_steps, n_inputs), y: y_test.reshape(-1, n_time_steps, n_outputs)})

In [ ]:
gru_mse

In [ ]:
testing_predictions = pd.Series(gru_predictions.ravel(),
                                index=testing_target.index,
                                name="predictions")
gru_results_df = pd.concat([testing_target, testing_predictions], axis=1)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10,8))
gru_results_df.head(60 * 24).plot(ax=ax);

In [ ]:
mean_absolute_percentage_error(gru_results_df.target, gru_results_df.predictions)